In [1]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

In [5]:
#weight tensor
W = tf.Variable(tf.zeros([784, 10], tf.float32))

#bias tensor
b = tf.Variable(tf.zeros([10], tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
sess.run(tf.global_variables_initializer())

In [7]:
tf.matmul(x,W) +  b

<tf.Tensor 'add:0' shape=(?, 10) dtype=float32>

In [8]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = [1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


In [10]:
#training batch
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict = {x: batch[0], y_: batch[1]})

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
acc = accuracy.eval(feed_dict = {x: mnist.test.images, y_: mnist.test.labels}) * 100
print("The final accuracy for the simple ANN model is: {} %".format(acc))

The final accuracy for the simple ANN model is: 91.3100004196167 %


In [13]:
sess.close()

In [29]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
###MNIST DATA using CNN
sess = tf.InteractiveSession()

#Initial parameters
width = 28
height = 28
flat = width * height
class_output = 10

C:\Users\sd16053\anaconda3\envs\ML\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [31]:
x = tf.placeholder(tf.float32, shape = [None, flat])
y_ = tf.placeholder(tf.float32, shape = [None, class_output])

In [32]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
#([batch_size, width, height, channels])
x_image

<tf.Tensor 'Reshape_2:0' shape=(?, 28, 28, 1) dtype=float32>

In [33]:
###Convolutional Layer1

In [34]:
w_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev = 0.1))
# [filter_hieght, filter_width, in_channels, out_channels]
b_conv1 = tf.Variable(tf.constant(0.1, shape = [32]))

In [35]:
convolve1 = tf.nn.conv2d(x_image, w_conv1, strides = [1, 1, 1, 1], padding = 'SAME') + b_conv1

In [36]:
h_conv1 = tf.nn.relu(convolve1)

In [37]:
conv1 = tf.nn.max_pool(h_conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') #max_pool_2x2
conv1

<tf.Tensor 'MaxPool_2:0' shape=(?, 14, 14, 32) dtype=float32>

In [38]:
###Convolutional Layer2

In [39]:
w_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.1))
# [filter_hieght, filter_width, in_channels, out_channels]
b_conv2 = tf.Variable(tf.constant(0.1, shape = [64])) #64 biases for 64 outputs

In [40]:
convolve2 = tf.nn.conv2d(conv1, w_conv2, strides = [1, 1, 1, 1], padding = 'SAME') + b_conv2

In [41]:
h_conv2 = tf.nn.relu(convolve2)

In [42]:
conv2 = tf.nn.max_pool(h_conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') #max_pool_2x2
conv2

<tf.Tensor 'MaxPool_3:0' shape=(?, 7, 7, 64) dtype=float32>

In [43]:
#flatten second layer
layer2_matrix = tf.reshape(conv2, [-1,7*7*64])

In [44]:
w_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev = 0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape = [1024])) 

In [45]:
fc1 = tf.matmul(layer2_matrix, w_fc1) + b_fc1

In [46]:
h_fc1 = tf.nn.relu(fc1)
h_fc1

<tf.Tensor 'Relu_5:0' shape=(?, 1024) dtype=float32>

In [47]:
#Dropout layer
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

In [48]:
w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev = 0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape = [10])) 

In [49]:
fc = tf.matmul(layer_drop, w_fc2) + b_fc2

In [50]:
y_CNN = tf.nn.softmax(fc)
y_CNN

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

In [51]:
#cross entropy
import numpy as np
layer4_test = [[0.9, 0.1, 0.1],[0.9,0.1,0.1]]
y_test = [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0]]
np.mean( -np.sum(y_test * np.log(layer4_test),1))

0.10536051565782628

In [52]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices = [1]))

In [53]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [54]:
correct_prediction = tf.equal(tf.argmax(y_CNN, 1), tf.argmax(y_, 1))

In [55]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [56]:
sess.run(tf.global_variables_initializer())

In [57]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("Step %d, training accuracy %g" %(i, float(train_accuracy)))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

Step 0, training accuracy 0.06
Step 100, training accuracy 0.86
Step 200, training accuracy 0.9
Step 300, training accuracy 0.94
Step 400, training accuracy 0.9
Step 500, training accuracy 0.98
Step 600, training accuracy 0.92
Step 700, training accuracy 0.94
Step 800, training accuracy 0.94
Step 900, training accuracy 0.96
Step 1000, training accuracy 0.98


In [59]:
# evaluate the model
n_batches = mnist.test.images.shape[0]//50
cumulative_accuracy = 0.0
for index in range(n_batches):
    batch = mnist.test.next_batch(50)
    cumulative_accuracy += accuracy.eval(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
print("test accuracy {}".format(cumulative_accuracy / n_batches))

test accuracy 0.9653999987244606


In [60]:
sess.close()